In [26]:
#imports
import datetime
from bs4 import BeautifulSoup
import requests

exo_planet_information = [] #list of dictionaries of exoplanet information, eventually to be filled up with scraped
# information from exoplanetarchive.ipac.caltech.edu

#example exoplanet (Kepler-186 f), each exoplanet information object will consist of the following:
kepler_186f = {
    'midpoint_julian': 2455789.4940,
    'duration': 0.2342,
    'orbital_period': 129.9441,
}

exo_planet_information.append(kepler_186f)

#this method is not yet filled, but by the final midterm project submission, will scrape most exoplanet information
#for the purposes of the draft, we will only be using info for kepler_186f
def scrape_all_exo_planet_information():
    pass

In [27]:
#julian date converter URL, pass in julian date value in 'jd=''. the resulting HTML page is scraped below
JULIAN_DATE_CONVERTER_BASE_URL = "https://aa.usno.navy.mil/jdconverter?ID=AA&jd="

#scrapes the resulting HTML page for the gregorian date, converts it into python datetime object
def convert_julian_days_to_legible_date(julian_date):
    
    #scrape the value of gregorian date from the site
    page = requests.get(JULIAN_DATE_CONVERTER_BASE_URL + str(julian_date))
    soup = BeautifulSoup(page.text, 'html.parser')
    tds = soup.find("table").find_all("tr")[1].find_all("td")
    
    #get date and time attributes
    date = tds[0].text
    time = tds[1].text
    
    #manipulate and combine
    period = date.index("D.")
    date = date[period + 3:]
    
    #turn into datetime object
    date_time_str = date + " " + time
    date_time_obj = datetime.datetime.strptime(date_time_str, '%Y %b %d %H:%M:%S.%f')
    
    return date_time_obj    

print(convert_julian_days_to_legible_date(2455789.4940)) #example

2011-08-15 23:51:21.600000


In [46]:
#methods below compute the next transit given the information of an exoplanet, the Kepler-186f example is used at the end
def compute_next_transit(exoplanet_info):
    transit_start = compute_transit_start_julian(exoplanet_info)
    transit_end = compute_transit_end_julian(exoplanet_info)
    
    delta = compute_transit_delta(exoplanet_info)
    now = datetime.datetime.now()
    
    while(convert_julian_days_to_legible_date(transit_start) < now):
        transit_start += delta
        transit_end += delta
    
    return (convert_julian_days_to_legible_date(transit_start).strftime('%Y %b %d %H:%M:%S.%f'),
            convert_julian_days_to_legible_date(transit_end).strftime('%Y %b %d %H:%M:%S.%f'))
    
def compute_transit_delta(exoplanet_info): 
    return exoplanet_info['orbital_period']

def compute_transit_start_julian(exo_planet_info):
    midpoint = exo_planet_info["midpoint_julian"]
    duration_to_midpoint = exo_planet_info["duration"] / 2
    
    start_time = midpoint - duration_to_midpoint
    
    return start_time

def compute_transit_end_julian(exo_planet_info):
    midpoint = exo_planet_info["midpoint_julian"]
    duration_to_midpoint = exo_planet_info["duration"] / 2
    
    start_time = midpoint + duration_to_midpoint
    
    return start_time

print("Next Transit(start,end):", compute_next_transit(exo_planet_information[0]))

Next Transit(start,end): ('2019 Jun 13 15:31:49.400000', '2019 Jun 13 21:09:04.300000')
